# Simulação de um Sistema Solar com um Vulcano

## Criando arquivos de entrada para simulação

In [18]:
# Import modules
import os
import sys
import yaml
import shutil
from glob import glob
import pandas as pd

## Definindo funções

In [19]:
def read_config_file(config_file):
    """
    Read config file and go to home directory
    """
    home_dir = os.path.dirname(os.path.abspath(config_file))
    os.chdir(home_dir)

    # Read yaml file
    with open(config_file, 'r') as stream:
        try:
            config = yaml.load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    
    return config

In [20]:
def write_pl(pl):
    """
    Write input files
    """
    for i, j in enumerate(glob("*")):
        os.chdir(j)
        with open("pl.in", "a") as f:
            f.write(str(pl.ix[i]["mass_grav"]) + "\n")
            f.write(str(pl.ix[i]["x"]) + " " + str(pl.ix[i]["y"]) + " " + str(pl.ix[i]["z"]) + "\n")
            f.write(str(pl.ix[i]["vx"]) + " " + str(pl.ix[i]["vy"]) + " " + str(pl.ix[i]["vz"]) + "\n")
        os.chdir("../")

## Leitura do arquivo de configuração e definição de variáveis

In [21]:
config_file = "config.yaml"
home_dir = os.path.dirname(os.path.abspath(config_file))
os.chdir(home_dir)

config = read_config_file("config.yaml")
simulation_name = config["simulation_name"]
planets_name = config['planets_names']
vulcan_variants_suffix = config["vulcans_semi_axis"]
vulcan_variants = len(config["vulcans_semi_axis"])
vulcan_clones = config["vulcans_clones"]
subroutine_swift = config["subroutine_swift"]

## Abrindo os dataframes 

### Vulcan

In [22]:
### Open planet's data
# vulcan
vulcan = pd.read_csv("vulcan.csv")

### Planetas

In [23]:
# Other planets
for i in planets_name:
    exec("{0} = pd.read_csv('{0}' + '.csv')".format(i))

## Criando os dados para inserir nos arquivos de configuração

### Sol

In [24]:
# ## Sun's data
sun_data = """10
2.959139768995959E-04
0.0 0.0 0.0
0.0 0.0 0.0
"""
os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open("pl.in", "a") as f:
        f.write(sun_data)
    os.chdir("../")

os.chdir(home_dir)

### Vulcan

In [25]:
# vulcan
os.chdir(simulation_name)

write_pl(vulcan)

os.chdir(home_dir)

### Outros planetas

In [26]:
# Others planets
os.chdir(simulation_name)
# write input files

for pl in planets_name:
    exec("write_pl({0})".format(pl))

os.chdir(home_dir)

### Partícula de testes

In [27]:
# ## Test particle
# There is no test particle
tp = """
0
"""

os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open("tp.in", "a") as f:
        f.write(tp)
    os.chdir("../")

os.chdir(home_dir)

### Parâmetros da simulação
param.in

In [28]:
# ## Parameters of the simulation
# 
# Write param.in
initial_time= config["initial_time"]
time_stop= config["time_stop"]
timestep= config["timestep"]
dtout= config["dtout"]
dtdump= config["dtdump"]
L1= config["L1"]
L2= config["L2"]
L3= config["L3"]
L4= config["L4"]
L5= config["L5"]
L6= config["L6"]
rmin= config["rmin"]
rmax= config["rmax"]
rmaxu= config["rmaxu"]
qmin= config["qmin"]
lclose= config["lclose"]
binary_outputfile= config["binary_outputfile"]
status_flag_for_open_statements= config["status_flag_for_open_statements"]

string = """{0} {1} {2}
{3} {4}
{5} {6} {7} {8} {9} {10}
{11} {12} {13} {14} {15}
{16}
{17}""".format(initial_time, time_stop, timestep, dtout, dtdump, \
                        L1, L2, L3, L4, L5, L6, rmin, rmax, rmaxu, qmin, \
                        lclose, binary_outputfile, status_flag_for_open_statements)

os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open("param.in", "a") as f:
        f.write(string)
    os.chdir("../")

os.chdir(home_dir)

### Argumento para a subrotina do SWIFT
swift_mvs.in

In [29]:
# ## arguments for Swift routine 
os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open(subroutine_swift, "a") as f:
        f.write("param.in\n")
        f.write("pl.in\n")
        f.write("tp.in")
    os.chdir("../")

os.chdir(home_dir)

## Verificando dados

### Dados no diretório de simulação

In [30]:
glob(simulation_name + "/" + simulation_name + "-A-001/*")

['vulcan_mvs/vulcan_mvs-A-001/pl.in',
 'vulcan_mvs/vulcan_mvs-A-001/tp.in',
 'vulcan_mvs/vulcan_mvs-A-001/param.in',
 'vulcan_mvs/vulcan_mvs-A-001/swift_mvs.in']

### pl.in

In [31]:
with open(simulation_name + "/" + simulation_name + "-A-001/pl.in") as f:
    print(f.read())

10
2.959139768995959E-04
0.0 0.0 0.0
0.0 0.0 0.0
8.88753856795e-10
0.0322228881946 -0.0388547956677 5.83736871443e-07
0.0585123570212 0.0488114573449 2.46262318524e-06
8.88753856795e-10
-0.160519238407 0.350467422863 -1.10953993424e-05
-0.0252031857258 -0.0116476605985 -6.53506426286e-07
7.24345628644e-10
0.612205881108 -0.380876360556 -8.1932729965e-05
0.0107356996565 0.0172183998229 -1.92933098973e-06
8.88753856795e-10
-0.886649473135 0.455132700562 -5.67839587112e-05
-0.00785902637308 -0.0153670206309 -6.17135597772e-07
9.5494572852e-11
1.2881115618 0.794766192946 1.77626251569e-05
-0.00740018866269 0.0119180685532 -1.32243838798e-07
2.82474705462e-07
-5.01564679347 -2.12533608925 0.12099712673
0.166031887915 -0.383099822888 -0.00212275165168
8.4576187896e-08
-1.01554741332 -9.99909627946 0.214273878795
0.304856193784 -0.0338073464286 -0.0115455453042
1.29188735613e-08
18.0772424721 8.38576561567 -0.203048827501
-0.0978886846842 0.196752743324 0.00199901665646
1.52403510768e-08
28.4

### tp.in

In [32]:
with open(simulation_name + "/" + simulation_name + "-A-001/tp.in") as f:
    print(f.read())


0



### param.in

In [33]:
with open(simulation_name + "/" + simulation_name + "-A-001/param.in") as f:
    print(f.read())

0.0e0 3.65249999999e8 0.2
3.6525e5 3.6525e5
F T F T T F
-1.0 1000.0 -1.0 -1.0 F
bin.out
unknown


### swift_mvs.in

In [34]:
with open(simulation_name + "/" + simulation_name + "-A-001/swift_mvs.in") as f:
    print(f.read())

param.in
pl.in
tp.in
